In [ ]:
# Setup
%pip install -e '../../.[kafka-python]'

import json
import uuid
from datetime import datetime

import requests
from diaspora_event_sdk import Client as GlobusClient
from diaspora_event_sdk.sdk.kafka_client import KafkaProducer

# import os
# os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()

In [ ]:
# Build HTTP headers for raw REST API calls
# The SDK's login_manager provides the authorizer with the access token.
from diaspora_event_sdk.sdk.login_manager import DiasporaScopes

authorizer = c.login_manager._get_authorizer(DiasporaScopes.resource_server)
base_url = c.web_service_address.rstrip("/")
ns = c.namespace

headers = {
    "Subject": c.subject_openid,
    "Authorization": f"Bearer {authorizer.access_token}",
    "Content-Type": "application/vnd.kafka.v2+json",
}

print(f"Base URL: {base_url}")
print(f"Namespace: {ns}")
print(f"Subject: {c.subject_openid}")

In [ ]:
# Create key + create topic (using SDK for initial setup)
key_result = c.create_key()
print(json.dumps(key_result, indent=2, default=str))

topic_name = f"topic-{str(uuid.uuid4())[:5]}"
create_topic_result = c.create_topic(topic_name)
print(f"\nCreated topic: {topic_name}")
print(json.dumps(create_topic_result, indent=2, default=str))

In [ ]:
# Produce 3 messages via KafkaProducer (seed data to consume via REST)
kafka_topic = f"{ns}.{topic_name}"
p = KafkaProducer(kafka_topic)
for i in range(3):
    message = {
        "message_id": i + 1,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "content": f"Message {i + 1} from ConsumerRESTDemo",
    }
    future = p.send(kafka_topic, message)
    result = future.get(timeout=30)
    print(f"Produced message {i + 1}: offset={result.offset}")
p.close()
print("\nAll messages produced.")

## Consumer REST API Walkthrough

All cells below use raw HTTP requests via the `requests` library,
matching the [Confluent Kafka REST Proxy v2 consumer API](https://docs.confluent.io/platform/current/kafka-rest/api.html#consumers).

No Kafka client or AWS credentials needed — only the Globus access token.

### Fetch records query parameters

| Parameter   | Type | Default | Description |
|-------------|------|---------|-------------|
| `timeout`   | int  | 5000    | Poll timeout in milliseconds |
| `max_bytes` | int  | None    | Maximum number of records to return (unlimited if omitted) |

In [ ]:
# 1. Create consumer instance
# POST /api/v3/{namespace}/consumers/{group_name}
resp = requests.post(
    f"{base_url}/api/v3/{ns}/consumers/demo-group",
    headers=headers,
    json={
        "name": "demo-inst",
        "format": "json",
        "auto.offset.reset": "earliest",
        "auto.commit.enable": "false",
    },
)
print(f"Status: {resp.status_code}")
data = resp.json()
print(json.dumps(data, indent=2))

instance_id = data["instance_id"]
consumer_base = f"{base_url}{data['base_uri']}"
print(f"\nConsumer base URI: {consumer_base}")

In [ ]:
# 2. Subscribe to topics
# POST .../subscription
resp = requests.post(
    f"{consumer_base}/subscription",
    headers=headers,
    json={"topics": [topic_name]},
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

In [ ]:
# 3. Get current subscription
# GET .../subscription
resp = requests.get(
    f"{consumer_base}/subscription",
    headers=headers,
)
print(f"Status: {resp.status_code}")
print(json.dumps(resp.json(), indent=2))

In [ ]:
# 4. Fetch records (1st poll)
# GET .../records?timeout=5000
# The first poll after subscribing may return empty
# (it triggers partition assignment).
resp = requests.get(
    f"{consumer_base}/records",
    headers=headers,
    params={"timeout": 5000},
)
print(f"Status: {resp.status_code}")
records = resp.json()
print(f"Records returned: {len(records)}")
if records:
    print(json.dumps(records, indent=2))

In [ ]:
# 5. Fetch records (2nd poll)
# GET .../records?timeout=5000
# This poll should return the messages.
resp = requests.get(
    f"{consumer_base}/records",
    headers=headers,
    params={"timeout": 5000},
)
print(f"Status: {resp.status_code}")
records = resp.json()
print(f"Records returned: {len(records)}")
print(json.dumps(records, indent=2))

In [ ]:
# 6. Commit offsets (commit all fetched records)
# POST .../offsets (empty body = commit all)
resp = requests.post(
    f"{consumer_base}/offsets",
    headers=headers,
)
print(f"Status: {resp.status_code}")

In [ ]:
# 7. Get committed offsets
# GET .../offsets (with partitions in request body)
resp = requests.get(
    f"{consumer_base}/offsets",
    headers=headers,
    json={"partitions": [{"topic": topic_name, "partition": 0}]},
)
print(f"Status: {resp.status_code}")
print(json.dumps(resp.json(), indent=2))

In [ ]:
# 8. Get partition assignments
# GET .../assignments
resp = requests.get(
    f"{consumer_base}/assignments",
    headers=headers,
)
print(f"Status: {resp.status_code}")
print(json.dumps(resp.json(), indent=2))

In [ ]:
# 9. Seek to beginning
# POST .../positions/beginning
resp = requests.post(
    f"{consumer_base}/positions/beginning",
    headers=headers,
    json={"partitions": [{"topic": topic_name, "partition": 0}]},
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

In [ ]:
# 10. Fetch records after seek to beginning (should re-read from offset 0)
# GET .../records?timeout=5000
resp = requests.get(
    f"{consumer_base}/records",
    headers=headers,
    params={"timeout": 5000},
)
print(f"Status: {resp.status_code}")
records = resp.json()
print(f"Records returned: {len(records)}")
print(json.dumps(records, indent=2))

In [ ]:
# 11. Seek to end
# POST .../positions/end
resp = requests.post(
    f"{consumer_base}/positions/end",
    headers=headers,
    json={"partitions": [{"topic": topic_name, "partition": 0}]},
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

In [ ]:
# 12. Seek to specific offset
# POST .../positions
resp = requests.post(
    f"{consumer_base}/positions",
    headers=headers,
    json={"offsets": [{"topic": topic_name, "partition": 0, "offset": 1}]},
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

# Fetch to verify — should start from offset 1 (skipping message 0)
resp = requests.get(
    f"{consumer_base}/records",
    headers=headers,
    params={"timeout": 5000},
)
print("\nRecords after seek to offset 1:")
records = resp.json()
print(f"Records returned: {len(records)}")
print(json.dumps(records, indent=2))

In [ ]:
# 13. Unsubscribe
# DELETE .../subscription
resp = requests.delete(
    f"{consumer_base}/subscription",
    headers=headers,
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

## Manual Partition Assignment

An alternative to `subscribe`. Instead of letting Kafka's group coordinator
assign partitions, you manually specify which topic-partitions to consume.

**Note**: `subscribe` and `assign` are mutually exclusive.

In [ ]:
# 14. Assign partitions manually
# POST .../assignments
resp = requests.post(
    f"{consumer_base}/assignments",
    headers=headers,
    json={"partitions": [{"topic": topic_name, "partition": 0}]},
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

In [ ]:
# 15. Get assignments (verify)
# GET .../assignments
resp = requests.get(
    f"{consumer_base}/assignments",
    headers=headers,
)
print(f"Status: {resp.status_code}")
print(json.dumps(resp.json(), indent=2))

In [ ]:
# 16. Seek to beginning and fetch after manual assignment
resp = requests.post(
    f"{consumer_base}/positions/beginning",
    headers=headers,
    json={"partitions": [{"topic": topic_name, "partition": 0}]},
)
print(f"Seek to beginning status: {resp.status_code}")

resp = requests.get(
    f"{consumer_base}/records",
    headers=headers,
    params={"timeout": 5000},
)
print(f"\nFetch status: {resp.status_code}")
records = resp.json()
print(f"Records returned: {len(records)}")
print(json.dumps(records, indent=2))

In [ ]:
# 17. Delete consumer instance
# DELETE .../instances/{instance_id}
resp = requests.delete(
    consumer_base,
    headers=headers,
)
print(f"Status: {resp.status_code}")
# Expect 204 No Content

## Cleanup

Delete the topic and access key created during setup.

In [ ]:
# Delete topic
delete_topic_result = c.delete_topic(topic_name)
print(json.dumps(delete_topic_result, indent=2, default=str))

# Delete key
delete_key_result = c.delete_key()
print(json.dumps(delete_key_result, indent=2, default=str))